# HW4
member:
1. 統計112 劉恩兆 B14081027
2. 統計112 宋穎恩 B14081032

# 載套件、讀資料

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import make_scorer

In [3]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [3]:
train

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,551,15806307,Trevisano,720,Spain,Male,38,5,114051.97,2,0,1,107577.29,0
1,6897,15709621,Martin,682,France,Female,54,4,62397.41,1,1,0,113088.60,1
2,4588,15619340,Palmer,672,France,Female,31,5,119903.67,1,1,1,132925.17,0
3,291,15620746,Napolitani,592,Spain,Female,40,4,104257.86,1,1,0,110857.33,0
4,1673,15646372,Yao,753,Spain,Male,42,5,120387.73,1,0,1,126378.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5345,15584532,Yu,568,France,Female,35,6,121079.60,2,1,1,124890.50,1
7996,5837,15606641,Liao,602,Germany,Female,45,7,145846.07,1,1,0,99276.02,0
7997,7335,15739692,Ferri,679,Spain,Female,43,5,132810.01,1,1,0,130780.85,1
7998,9552,15791373,Worsnop,715,France,Male,38,4,118729.45,1,0,0,95484.52,0


In [6]:
test

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,2209,15623595,Scott,726,France,Female,39,3,62397.41,1,0,0,106139.31
1,9924,15740283,Yegorova,661,France,Male,42,3,130339.64,1,1,0,125776.38
2,4617,15802368,Tan,545,France,Male,34,6,62397.41,2,1,1,78372.28
3,6077,15615322,Anayochukwu,731,Spain,Female,42,3,86717.08,1,1,0,136026.27
4,9240,15766374,Glover,533,France,Female,50,6,62397.41,1,1,1,135205.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,7872,15617348,Taylor,628,France,Male,35,6,62397.41,2,1,1,96201.09
1996,4257,15642785,Tokareva,639,Germany,Male,41,5,98154.30,2,1,0,109531.64
1997,2273,15766575,Trevisan,548,France,Female,46,3,119078.34,2,1,1,116725.67
1998,315,15610271,Dickson,602,Spain,Female,44,7,62397.41,2,1,1,128135.95


# 資料前處理

In [3]:
train.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)
RowNumber = test['RowNumber']
test.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)

In [4]:
train.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
test.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [5]:
#【train】
#Standard Scaler
from sklearn.preprocessing import StandardScaler
col_to_scale = ['CreditScore', 'Balance', 'EstimatedSalary']
scaler = StandardScaler().fit(train[col_to_scale])
train_scaled = scaler.transform(train[col_to_scale])
train[col_to_scale] = train_scaled
#One-hot encoding
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=['Geography'])
train = encoder.fit_transform(train)
#Gender map
train['Gender'] = train['Gender'].map({'Female':0, 'Male':1})
train

,CreditScore,Geography_1,Geography_2,Geography_3,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.509906,1,0,0,1,38,5,0.428978,2,0,1,0.304927,0
1,0.866637,0,1,0,0,54,4,-1.214512,1,1,0,0.529787,1
2,0.697356,0,1,0,0,31,5,0.615161,1,1,1,1.339115,0
3,-0.656894,1,0,0,0,40,4,0.117359,1,1,0,0.438752,0
4,2.068534,1,0,0,1,42,5,0.630562,1,0,1,1.072015,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-1.063169,0,1,0,0,35,6,0.652575,2,1,1,1.011302,1
7996,-0.487612,0,0,1,0,45,7,1.440568,1,1,0,-0.033763,0
7997,0.815853,1,0,0,0,43,5,1.025801,1,1,0,1.251627,1
7998,1.425266,0,1,0,1,38,4,0.577801,1,0,0,-0.188456,0


In [6]:
#【test】
#Standard Scaler 
from sklearn.preprocessing import StandardScaler
col_to_scale = ['CreditScore', 'Balance', 'EstimatedSalary']
scaler = StandardScaler().fit(test[col_to_scale])
test_scaled = scaler.transform(test[col_to_scale])
test[col_to_scale] = test_scaled
#One-hot encoding
encoder = ce.OneHotEncoder(cols=['Geography'])
test = encoder.fit_transform(test)
#Gender map
test['Gender'] = test['Gender'].map({'Female':0, 'Male':1})
test

,CreditScore,Geography_1,Geography_2,Geography_3,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1.609183,1,0,0,0,39,3,-1.272730,1,0,0,0.263584
1,0.528128,1,0,0,1,42,3,0.915085,1,1,0,1.077935
2,-1.401138,1,0,0,1,34,6,-1.272730,2,1,1,-0.887917
3,1.692341,0,1,0,0,42,3,-0.489610,1,1,0,1.502999
4,-1.600717,1,0,0,0,50,6,-1.272730,1,1,1,1.468965
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.020715,1,0,0,1,35,6,-1.272730,2,1,1,-0.148555
1996,0.162233,0,0,1,1,41,5,-0.121318,2,1,0,0.404264
1997,-1.351243,1,0,0,0,46,3,0.552459,2,1,1,0.702602
1998,-0.453136,0,1,0,0,44,7,-1.272730,2,1,1,1.175787


In [7]:
train_X = train.drop(['Exited'], axis=1)
train_Y = train['Exited']
test_X = test

# 跑模型

## LogisticRegression (1)

In [7]:
from sklearn.linear_model import LogisticRegression

# 建立Logistic模型
logisticModel = LogisticRegression(random_state=0, max_iter=1000, verbose=1)
# 使用訓練資料訓練模型
logisticModel.fit(train_X, train_Y.astype(int))
# 使用訓練資料預測分類
ypred = logisticModel.predict(test_X)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [19]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_1 = pd.concat([RowNumber,Y_pred], axis=1)
upload_1.rename(columns={0: 'Exited'}, inplace=True)
upload_1

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,0
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [20]:
upload_1.to_csv('upload_1.csv')

## SVM (2)

In [30]:
from sklearn.svm import SVC
model = SVC(max_iter=1000, C=1)
model.fit(train_X, train_Y.astype(int))
ypred = model.predict(test_X)

C:\Anaconda\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [31]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_2 = pd.concat([RowNumber,Y_pred], axis=1)
upload_2.rename(columns={0: 'Exited'}, inplace=True)
upload_2

,RowNumber,Exited
0,2209,1
1,9924,1
2,4617,0
3,6077,1
4,9240,1
...,...,...
1995,7872,0
1996,4257,1
1997,2273,1
1998,315,1


In [23]:
upload_2.to_csv('upload_2.csv')

## MLP (3、59-66)

In [33]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'tanh', alpha= 0.0005, hidden_layer_sizes= (20, 15), learning_rate= 'constant', max_iter= 2000, solver= 'adam')
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)

In [34]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_3 = pd.concat([RowNumber,Y_pred], axis=1)
upload_3.rename(columns={0: 'Exited'}, inplace=True)
upload_3

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,0
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [35]:
upload_3.to_csv('upload_3.csv')

In [21]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'tanh', alpha= 0.0005, hidden_layer_sizes= (20, 15), learning_rate='constant', max_iter= 1000, solver= 'adam',learning_rate_init=0.005)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload66.csv')

In [15]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'relu', alpha= 0.0005, hidden_layer_sizes= (20, 15), learning_rate='constant', max_iter= 1000, solver= 'adam',learning_rate_init=0.005)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload59.csv')

In [15]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'relu', alpha= 0.0005, hidden_layer_sizes= (20, 15), learning_rate='constant', max_iter= 1000, solver= 'adam',learning_rate_init=0.01)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload60.csv')

In [16]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'relu', alpha= 0.0005, hidden_layer_sizes= (40, 20, 15), learning_rate='constant', max_iter= 1000, solver= 'adam',learning_rate_init=0.01)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload61.csv')

In [17]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'relu', alpha= 0.0005, hidden_layer_sizes= (40, 20, 15), learning_rate='constant', max_iter= 2000, solver= 'adam',learning_rate_init=0.005)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload62.csv')

In [18]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'relu', alpha= 0.0005, hidden_layer_sizes= (60, 40, 20), learning_rate='constant', max_iter= 1000, solver= 'adam',learning_rate_init=0.01)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload63.csv')

In [18]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'relu', alpha= 0.0005, hidden_layer_sizes= (60, 40, 20), learning_rate='constant', max_iter= 1000, solver= 'adam',learning_rate_init=0.01)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload63.csv')

In [19]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'tanh', alpha= 0.0005, hidden_layer_sizes= (40, 20, 15), learning_rate='constant', max_iter= 1000, solver= 'adam',learning_rate_init=0.01)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload64.csv')

In [20]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(activation= 'relu', alpha= 0.0005, hidden_layer_sizes= (40, 20, 15), learning_rate='adaptive', max_iter= 1000, solver= 'adam',learning_rate_init=0.01)
clf.fit(train_X, train_Y.astype(int))
ypred=clf.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload65.csv')

## Decision Tree (4、5)

In [36]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(train_X, train_Y.astype(int))
ypred = dt.predict(test_X)

In [37]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_4 = pd.concat([RowNumber,Y_pred], axis=1)
upload_4.rename(columns={0: 'Exited'}, inplace=True)
upload_4

,RowNumber,Exited
0,2209,1
1,9924,0
2,4617,0
3,6077,0
4,9240,0
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [38]:
upload_4.to_csv('upload_4.csv')

In [39]:
dt = DecisionTreeClassifier(max_depth = 6, min_samples_leaf=4, min_samples_split=20)
dt.fit(train_X, train_Y.astype(int))
ypred = dt.predict(test_X)

In [40]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_5 = pd.concat([RowNumber,Y_pred], axis=1)
upload_5.rename(columns={0: 'Exited'}, inplace=True)
upload_5

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,0
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [41]:
upload_5.to_csv('upload_5.csv')

## KNN (6、7)

In [42]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn = knn.fit(train_X, train_Y.astype(int))
ypred = knn.predict(test_X)

C:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [43]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_6 = pd.concat([RowNumber,Y_pred], axis=1)
upload_6.rename(columns={0: 'Exited'}, inplace=True)
upload_6

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,1
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [44]:
upload_6.to_csv('upload_6.csv')

In [45]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(algorithm='ball_tree', n_neighbors=50, p=1)
knn = knn.fit(train_X, train_Y.astype(int))
ypred = knn.predict(test_X)

C:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [46]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_7 = pd.concat([RowNumber,Y_pred], axis=1)
upload_7.rename(columns={0: 'Exited'}, inplace=True)
upload_7

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,0
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [47]:
upload_7.to_csv('upload_7.csv')

## Random Forest (8、9)

In [53]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train_X, train_Y.astype(int))
ypred = rf.predict(test_X)

In [54]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_8 = pd.concat([RowNumber,Y_pred], axis=1)
upload_8.rename(columns={0: 'Exited'}, inplace=True)
upload_8

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,0
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [55]:
upload_8.to_csv('upload_8.csv')

In [49]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth = 6, min_samples_leaf=4, min_samples_split=20)
rf.fit(train_X, train_Y.astype(int))
ypred = rf.predict(test_X)

In [50]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_8 = pd.concat([RowNumber,Y_pred], axis=1)
upload_8.rename(columns={0: 'Exited'}, inplace=True)
upload_8

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,0
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [52]:
upload_9.to_csv('upload_9.csv')

## Gradient Boost (10、14-35)

In [57]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)

In [58]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_10 = pd.concat([RowNumber,Y_pred], axis=1)
upload_10.rename(columns={0: 'Exited'}, inplace=True)
upload_10

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,1
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [59]:
upload_10.to_csv('upload_10.csv')

### 最佳化參數

#### learning rate

In [9]:
#14 lr=0.001
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.001)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_14.csv')

In [10]:
#15 lr=0.01
gb = GradientBoostingClassifier(learning_rate=0.01)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_15.csv')

In [11]:
#16 lr=0.05
gb = GradientBoostingClassifier(learning_rate=0.05)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_16.csv')

In [12]:
#17 lr=0.1
gb = GradientBoostingClassifier(learning_rate=0.1)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_17.csv')

In [13]:
#18 lr=0.25
gb = GradientBoostingClassifier(learning_rate=0.25)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_18.csv')

#### n_estimators

In [15]:
#20 lr=0.25, n_estimators=10
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=10)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_20.csv')

In [16]:
#21 lr=0.25, n_estimators=100
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_21.csv')

In [17]:
#22 lr=0.25, n_estimators=500
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=500)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_22.csv')

In [18]:
#23 lr=0.25, n_estimators=1000
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=1000)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_23.csv')

In [19]:
#24 lr=0.25, n_estimators=5000
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=5000)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_24.csv')

In [20]:
#25 lr=0.25, n_estimators=50
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=10)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_25.csv')

#### min_samples_leaf

In [21]:
#26 lr=0.25, n_estimators=100, min_samples_leaf=30
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100, min_samples_leaf=30)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_26.csv')

In [25]:
#27 lr=0.25, n_estimators=100, min_samples_leaf=40
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100, min_samples_leaf=40)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_27.csv')

In [22]:
#28 lr=0.25, n_estimators=100, min_samples_leaf=50
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100, min_samples_leaf=50)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_28.csv')

In [23]:
#29 lr=0.25, n_estimators=100, min_samples_leaf=60
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100, min_samples_leaf=30)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_29.csv')

In [24]:
#30 lr=0.25, n_estimators=100, min_samples_leaf=70
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100, min_samples_leaf=70)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_30.csv')

#### max_depth

In [26]:
#31 lr=0.25, n_estimators=100, max_depth=1
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100,max_depth=1)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_31.csv')

In [29]:
#32 lr=0.25, n_estimators=100, max_depth=3
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100,max_depth=3)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_32.csv')

In [30]:
#33 lr=0.25, n_estimators=100, max_depth=5
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100,max_depth=5)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_33.csv')

In [31]:
#34 lr=0.25, n_estimators=100, max_depth=7
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100,max_depth=7)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_34.csv')

In [32]:
#35 lr=0.25, n_estimators=100, max_depth=9
gb = GradientBoostingClassifier(learning_rate=0.25,n_estimators=100,max_depth=9)
gb.fit(train_X, train_Y.astype(int))
ypred = gb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_35.csv')

## XGBoost (11、67-71)

In [62]:
from xgboost import  XGBClassifier
xgb = XGBClassifier()
xgb.fit(train_X, train_Y.astype(int))
ypred = xgb.predict(test_X)

In [63]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_11 = pd.concat([RowNumber,Y_pred], axis=1)
upload_11.rename(columns={0: 'Exited'}, inplace=True)
upload_11

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,1
...,...,...
1995,7872,0
1996,4257,0
1997,2273,1
1998,315,0


In [64]:
upload_11.to_csv('upload_11.csv')

In [26]:
from xgboost import  XGBClassifier
xgb = XGBClassifier(learning_rate = 0.01, n_estimators=1000, gamma=0.1)
xgb.fit(train_X, train_Y.astype(int))
ypred = xgb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_67.csv')

In [27]:
from xgboost import  XGBClassifier
xgb = XGBClassifier(learning_rate = 0.05, n_estimators=1000, gamma=0.1)
xgb.fit(train_X, train_Y.astype(int))
ypred = xgb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_68.csv')

In [28]:
from xgboost import  XGBClassifier
xgb = XGBClassifier(learning_rate = 0.01, n_estimators=1000, gamma=0.3)
xgb.fit(train_X, train_Y.astype(int))
ypred = xgb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_69.csv')

In [29]:
from xgboost import  XGBClassifier
xgb = XGBClassifier(learning_rate = 0.01, n_estimators=1000, gamma=0.1, max_depth=9)
xgb.fit(train_X, train_Y.astype(int))
ypred = xgb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_70.csv')

In [30]:
from xgboost import  XGBClassifier
xgb = XGBClassifier(learning_rate = 0.01, n_estimators=1000, gamma=0.1, max_depth=13)
xgb.fit(train_X, train_Y.astype(int))
ypred = xgb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_71.csv')

## LGBM (12、45-58)

In [67]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)

In [68]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_12 = pd.concat([RowNumber,Y_pred], axis=1)
upload_12.rename(columns={0: 'Exited'}, inplace=True)
upload_12

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,0
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [69]:
upload_12.to_csv('upload_12.csv')

In [26]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 1000,learning_rate = 0.05) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_45.csv')

In [27]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 1000,learning_rate = 0.1) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_46.csv')

In [28]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 1000,learning_rate = 0.01) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_47.csv')

In [29]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 1000,learning_rate = 0.005) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_48.csv')

In [30]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 1000,learning_rate = 0.001) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_49.csv')

In [32]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 1000,learning_rate = 0.0005) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_50.csv')

In [33]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 300,learning_rate = 0.001) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_51.csv')

In [34]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 300,learning_rate = 0.005) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_52.csv')

In [38]:
upload_50 = pd.read_csv('upload_50.csv')
upload_48 = pd.read_csv('upload_48.csv')
for i in range(len(upload_48)):
    if upload_50['Exited'][i] == 1:
        print(i)
        upload_48['Exited'][i] = 1
upload_48.to_csv('upload_53.csv')

98
768
995
1374
1398
1831
1858


In [8]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 300,learning_rate = 0.005, max_bin=255) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_54.csv')

In [9]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 300,learning_rate = 0.005, max_bin=255, max_depth=4) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_55.csv')

In [10]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 300,learning_rate = 0.005, max_bin=255, max_depth=6) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_56.csv')

In [11]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 1500,learning_rate = 0.005) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_57.csv')

In [12]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
classifier = lgb.LGBMClassifier(objective = 'binary', random_state=0,n_estimators = 800,learning_rate = 0.005) 
# learning_rate = 0.05, 
# n_estimators = 100
classifier.fit(train_X, train_Y.astype(int))
ypred = classifier.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_58.csv')

## CatBoost (13)

In [71]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier()

# iterations=5, 
# learning_rate=0.1, 
#loss_function='CrossEntropy'

clf.fit(train_X, train_Y.astype(int), verbose=False)
# cat_features=cat_features, 
# eval_set=(X_val, y_val), 

ypred = clf.predict(test_X)

In [72]:
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload_13 = pd.concat([RowNumber,Y_pred], axis=1)
upload_13.rename(columns={0: 'Exited'}, inplace=True)
upload_13

,RowNumber,Exited
0,2209,0
1,9924,0
2,4617,0
3,6077,0
4,9240,1
...,...,...
1995,7872,0
1996,4257,0
1997,2273,0
1998,315,0


In [73]:
upload_13.to_csv('upload_13.csv')

## Neural Network (36-130)

In [18]:
from keras.models import Sequential
from keras.layers import Dense
 
model = Sequential() 
model.add(Dense(128, activation='relu', input_dim=12))
model.add(Dense(64, activation='relu')) 
model.add(Dense(16, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 128)               1664      
                                                                 
 dense_14 (Dense)            (None, 64)                8256      
                                                                 
 dense_15 (Dense)            (None, 16)                1040      
                                                                 
 dense_16 (Dense)            (None, 1)                 17        
                                                                 
Total params: 10,977
Trainable params: 10,977
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(train_X, train_Y.astype(int), epochs=200, batch_size=100)

Epoch 1/200
80/80 [==============================] - 2s 5ms/step - loss: 0.7331 - accuracy: 0.7579
Epoch 2/200
80/80 [==============================] - 0s 4ms/step - loss: 0.5190 - accuracy: 0.7960
Epoch 3/200
80/80 [==============================] - 0s 3ms/step - loss: 0.4962 - accuracy: 0.7960
Epoch 4/200
80/80 [==============================] - 0s 4ms/step - loss: 0.4755 - accuracy: 0.7986
Epoch 5/200
80/80 [==============================] - 0s 4ms/step - loss: 0.4623 - accuracy: 0.7989
Epoch 6/200
80/80 [==============================] - 0s 4ms/step - loss: 0.4531 - accuracy: 0.8014
Epoch 7/200
80/80 [==============================] - 0s 4ms/step - loss: 0.4535 - accuracy: 0.8061
Epoch 8/200
80/80 [==============================] - 0s 4ms/step - loss: 0.4497 - accuracy: 0.8037
Epoch 9/200
80/80 [==============================] - 0s 4ms/step - loss: 0.4468 - accuracy: 0.8061
Epoch 10/200
80/80 [==============================] - 0s 3ms/step - loss: 0.4443 - accuracy: 0.8076
Epoch 11/

In [17]:
ypred = np.round(model.predict(test_X),0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_36.csv')

63/63 [==============================] - 0s 2ms/step


In [19]:
ypred = np.where(model.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_37.csv')

63/63 [==============================] - 0s 2ms/step


In [20]:
ypred = np.where(model.predict(test_X)>0.58,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_43.csv')

63/63 [==============================] - 0s 2ms/step


In [23]:
model.fit(train_X, train_Y.astype(int), epochs=200, batch_size=100)

Epoch 1/200
80/80 [==============================] - 1s 2ms/step - loss: 0.5130 - accuracy: 0.7908
Epoch 2/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.7976
Epoch 3/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4697 - accuracy: 0.8019
Epoch 4/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4581 - accuracy: 0.7989
Epoch 5/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4456 - accuracy: 0.8069
Epoch 6/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4497 - accuracy: 0.8073
Epoch 7/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4498 - accuracy: 0.8066
Epoch 8/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4472 - accuracy: 0.8076
Epoch 9/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4379 - accuracy: 0.8120
Epoch 10/200
80/80 [==============================] - 0s 2ms/step - loss: 0.4359 - accuracy: 0.8129
Epoch 11/

In [24]:
ypred = np.where(model.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_39.csv')

63/63 [==============================] - 0s 2ms/step


In [7]:
from keras.models import Sequential
from keras.layers import Dense
 
model = Sequential() 
model.add(Dense(128, activation='relu', input_dim=12))
model.add(Dense(64, activation='relu')) 
model.add(Dense(32, activation='relu')) 
model.add(Dense(16, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1664      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 1)                 17        
                                                                 
Total params: 12,545
Trainable params: 12,545
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(train_X, train_Y.astype(int), epochs=300, batch_size=100)

Epoch 1/300
80/80 [==============================] - 2s 4ms/step - loss: 0.5181 - accuracy: 0.7958
Epoch 2/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4820 - accuracy: 0.7962
Epoch 3/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4667 - accuracy: 0.7996
Epoch 4/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4549 - accuracy: 0.8039
Epoch 5/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4483 - accuracy: 0.8059
Epoch 6/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4531 - accuracy: 0.8036
Epoch 7/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4442 - accuracy: 0.8079
Epoch 8/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4403 - accuracy: 0.8065
Epoch 9/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4334 - accuracy: 0.8096
Epoch 10/300
80/80 [==============================] - 0s 4ms/step - loss: 0.4279 - accuracy: 0.8119
Epoch 11/

In [9]:
ypred = np.where(model.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_40.csv')

63/63 [==============================] - 0s 2ms/step


In [10]:
from keras.models import Sequential
from keras.layers import Dense
 
model = Sequential() 
model.add(Dense(128, activation='relu', input_dim=12))
model.add(Dense(64, activation='relu')) 
model.add(Dense(32, activation='relu')) 
model.add(Dense(16, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               1664      
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 12,545
Trainable params: 12,545
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(train_X, train_Y.astype(int), epochs=300, batch_size=10)

Epoch 1/300
800/800 [==============================] - 2s 3ms/step - loss: 0.4557 - accuracy: 0.8034
Epoch 2/300
800/800 [==============================] - 2s 3ms/step - loss: 0.4508 - accuracy: 0.8052
Epoch 3/300
800/800 [==============================] - 2s 3ms/step - loss: 0.4427 - accuracy: 0.8111
Epoch 4/300
800/800 [==============================] - 2s 3ms/step - loss: 0.4380 - accuracy: 0.8110
Epoch 5/300
800/800 [==============================] - 2s 3ms/step - loss: 0.4308 - accuracy: 0.8125
Epoch 6/300
800/800 [==============================] - 2s 3ms/step - loss: 0.4238 - accuracy: 0.8144
Epoch 7/300
800/800 [==============================] - 2s 3ms/step - loss: 0.4051 - accuracy: 0.8204
Epoch 8/300
800/800 [==============================] - 2s 3ms/step - loss: 0.3963 - accuracy: 0.8284
Epoch 9/300
800/800 [==============================] - 2s 3ms/step - loss: 0.3876 - accuracy: 0.8317
Epoch 10/300
800/800 [==============================] - 2s 3ms/step - loss: 0.3765 - accura

In [13]:
ypred = np.where(model.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_41.csv')

63/63 [==============================] - 0s 2ms/step


In [14]:
from keras.models import Sequential
from keras.layers import Dense
 
model = Sequential() 
model.add(Dense(64, activation='relu', input_dim=12))
model.add(Dense(16, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                832       
                                                                 
 dense_11 (Dense)            (None, 16)                1040      
                                                                 
 dense_12 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1,889
Trainable params: 1,889
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_X, train_Y.astype(int), epochs=300, batch_size=100)

Epoch 1/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4460 - accuracy: 0.8095
Epoch 2/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4434 - accuracy: 0.8061
Epoch 3/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4442 - accuracy: 0.8089
Epoch 4/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4490 - accuracy: 0.8040
Epoch 5/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4457 - accuracy: 0.8061
Epoch 6/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4389 - accuracy: 0.8119
Epoch 7/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4386 - accuracy: 0.8120
Epoch 8/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4379 - accuracy: 0.8094
Epoch 9/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4393 - accuracy: 0.8102
Epoch 10/300
80/80 [==============================] - 0s 3ms/step - loss: 0.4365 - accuracy: 0.8121
Epoch 11/

In [17]:
ypred = np.where(model.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_42.csv')

63/63 [==============================] - 0s 2ms/step


In [22]:
from tensorflow.keras.layers import BatchNormalization
model = Sequential() 
model.add(Dense(128, activation='relu', input_dim=12))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 128)               1664      
                                                                 
 batch_normalization (BatchN  (None, 128)              512       
 ormalization)                                                   
                                                                 
 dense_19 (Dense)            (None, 64)                8256      
                                                                 
 batch_normalization_1 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 dense_20 (Dense)            (None, 16)                1040      
                                                                 
 batch_normalization_2 (Batc  (None, 16)              

In [23]:
model.fit(train_X, train_Y.astype(int), epochs=200, batch_size=100)

Epoch 1/200
80/80 [==============================] - 3s 5ms/step - loss: 0.5951 - accuracy: 0.6977
Epoch 2/200
80/80 [==============================] - 0s 5ms/step - loss: 0.4507 - accuracy: 0.8275
Epoch 3/200
80/80 [==============================] - 0s 5ms/step - loss: 0.3857 - accuracy: 0.8490
Epoch 4/200
80/80 [==============================] - 0s 4ms/step - loss: 0.3575 - accuracy: 0.8584
Epoch 5/200
80/80 [==============================] - 0s 5ms/step - loss: 0.3433 - accuracy: 0.8599
Epoch 6/200
80/80 [==============================] - 0s 5ms/step - loss: 0.3410 - accuracy: 0.8624
Epoch 7/200
80/80 [==============================] - 0s 5ms/step - loss: 0.3397 - accuracy: 0.8605
Epoch 8/200
80/80 [==============================] - 0s 5ms/step - loss: 0.3343 - accuracy: 0.8606
Epoch 9/200
80/80 [==============================] - 0s 4ms/step - loss: 0.3317 - accuracy: 0.8625
Epoch 10/200
80/80 [==============================] - 0s 4ms/step - loss: 0.3303 - accuracy: 0.8609
Epoch 11/

In [17]:
ypred = np.round(model.predict(test_X),0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_36.csv')

63/63 [==============================] - 0s 2ms/step


In [24]:
ypred = np.where(model.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_44.csv')

63/63 [==============================] - 0s 3ms/step


#### 72

In [13]:
from xgboost import XGBClassifier

model_xgb=XGBClassifier(n_jobs=-1,random_state=42,eval_metric='logloss')
parameters = {'n_estimators':[50,100,200, 300],'max_depth':[3,4,5,6,7],'learning_rate':[0.1,0.01,0.05,0.001]}
model_xgb_grid = GridSearchCV(model_xgb, parameters,cv=10,verbose=1).fit(train_X,train_Y)
print(model_xgb_grid.best_params_)

Fitting 10 folds for each of 80 candidates, totalling 800 fits
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200}


In [15]:
model_xgb=XGBClassifier(n_jobs=-1,random_state=42,n_estimators=200,max_depth=4,learning_rate=0.1,eval_metric='logloss')
model_xgb.fit(train_X,train_Y)
ypred = model_xgb.predict(test_X)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload
upload.to_csv('upload_72.csv')

#### 73-81

In [20]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [21]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [22]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_73.csv')

63/63 [==============================] - 0s 488us/step


In [23]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_74.csv')

63/63 [==============================] - 0s 625us/step


In [24]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_75.csv')

63/63 [==============================] - 0s 456us/step


In [25]:
ypred = np.where(model_dnn.predict(test_X)>0.44,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_76.csv')

63/63 [==============================] - 0s 546us/step


In [26]:
ypred = np.where(model_dnn.predict(test_X)>0.45,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_77.csv')

63/63 [==============================] - 0s 726us/step


In [28]:
ypred = np.where(model_dnn.predict(test_X)>0.448,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_78.csv')

63/63 [==============================] - 0s 756us/step


In [29]:
ypred = np.where(model_dnn.predict(test_X)>0.446,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_79.csv')

63/63 [==============================] - 0s 510us/step


In [30]:
ypred = np.where(model_dnn.predict(test_X)>0.452,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_80.csv')

63/63 [==============================] - 0s 711us/step


In [31]:
ypred = np.where(model_dnn.predict(test_X)>0.454,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_81.csv')

63/63 [==============================] - 0s 596us/step


#### 82

In [32]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),loss="binary_crossentropy",metrics=['accuracy'])

In [33]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [34]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_82.csv')

63/63 [==============================] - 0s 2ms/step


#### 83-85

In [35]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(3,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [36]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [37]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_83.csv')

63/63 [==============================] - 0s 2ms/step


In [38]:
ypred = np.where(model_dnn.predict(test_X)>0.45,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_84.csv')

63/63 [==============================] - 0s 2ms/step


In [39]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_85.csv')

63/63 [==============================] - 0s 2ms/step


#### 86-88

In [40]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(12,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [41]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [42]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_86.csv')

63/63 [==============================] - 0s 2ms/step


In [43]:
ypred = np.where(model_dnn.predict(test_X)>0.45,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_87.csv')

63/63 [==============================] - 0s 2ms/step


In [44]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_88.csv')

63/63 [==============================] - 0s 2ms/step


#### 89-91

In [45]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.4))
model_dnn.add(tf.keras.layers.Dense(9,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [46]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [47]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_89.csv')

63/63 [==============================] - 0s 2ms/step


In [48]:
ypred = np.where(model_dnn.predict(test_X)>0.45,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_90.csv')

63/63 [==============================] - 0s 2ms/step


In [49]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_91.csv')

63/63 [==============================] - 0s 2ms/step


#### 92-94

In [8]:
train_X_2 = train_X.copy()
train_Y_2 = train_Y.copy()
train_X_3 = pd.concat([train_X_2, train_X_2], axis = 0)
train_Y_3 = pd.concat([train_Y_2, train_Y_2], axis = 0)
train_X_3

,CreditScore,Geography_1,Geography_2,Geography_3,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1.509906,1,0,0,1,-0.663721,5,0.428978,0.799546,0,1,0.304927
1,0.866637,0,1,0,0,1.465241,4,-1.214512,-0.917139,1,0,0.529787
2,0.697356,0,1,0,0,-1.595141,5,0.615161,-0.917139,1,1,1.339115
3,-0.656894,1,0,0,0,-0.397600,4,0.117359,-0.917139,1,0,0.438752
4,2.068534,1,0,0,1,-0.131480,5,0.630562,-0.917139,0,1,1.072015
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-1.063169,0,1,0,0,-1.062901,6,0.652575,0.799546,1,1,1.011302
7996,-0.487612,0,0,1,0,0.267700,7,1.440568,-0.917139,1,0,-0.033763
7997,0.815853,1,0,0,0,0.001580,5,1.025801,-0.917139,1,0,1.251627
7998,1.425266,0,1,0,1,-0.663721,4,0.577801,-0.917139,0,0,-0.188456


In [10]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),loss="binary_crossentropy",metrics=['accuracy'])

In [11]:
model_dnn.fit(train_X_3,train_Y_3,epochs=200,verbose=0)

In [12]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_92.csv')

63/63 [==============================] - 0s 2ms/step


In [13]:
ypred = np.where(model_dnn.predict(test_X)>0.45,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_93.csv')

63/63 [==============================] - 0s 2ms/step


In [14]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_94.csv')

63/63 [==============================] - 0s 2ms/step


#### 95-97

In [15]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(30,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [17]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [18]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_95.csv')

63/63 [==============================] - 0s 2ms/step


In [19]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_96.csv')

63/63 [==============================] - 0s 2ms/step


In [20]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_97.csv')

63/63 [==============================] - 0s 2ms/step


#### 98-100

In [21]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(20,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [22]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [23]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_98.csv')

63/63 [==============================] - 0s 2ms/step


In [24]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_99.csv')

63/63 [==============================] - 0s 2ms/step


In [25]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_100.csv')

63/63 [==============================] - 0s 2ms/step


#### 101-108

In [26]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(20,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(3,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.2))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [27]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [28]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_101.csv')

63/63 [==============================] - 0s 2ms/step


In [29]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_102.csv')

63/63 [==============================] - 0s 2ms/step


In [30]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_103.csv')

63/63 [==============================] - 0s 2ms/step


In [31]:
ypred = np.where(model_dnn.predict(test_X)>0.42,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_104.csv')

63/63 [==============================] - 0s 2ms/step


In [32]:
ypred = np.where(model_dnn.predict(test_X)>0.40,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_105.csv')

63/63 [==============================] - 0s 2ms/step


In [33]:
ypred = np.where(model_dnn.predict(test_X)>0.385,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_106.csv')

63/63 [==============================] - 0s 2ms/step


In [34]:
ypred = np.where(model_dnn.predict(test_X)>0.39,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_107.csv')

63/63 [==============================] - 0s 2ms/step


In [35]:
ypred = np.where(model_dnn.predict(test_X)>0.395,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_108.csv')

63/63 [==============================] - 0s 2ms/step


#### 109-111

In [36]:
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(15,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.2))
model_dnn.add(tf.keras.layers.Dense(7,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.2))
model_dnn.add(tf.keras.layers.Dense(7,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.2))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [37]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [38]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_109.csv')

63/63 [==============================] - 0s 2ms/step


In [39]:
ypred = np.where(model_dnn.predict(test_X)>0.48,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_110.csv')

63/63 [==============================] - 0s 2ms/step


In [40]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_111.csv')

63/63 [==============================] - 0s 2ms/step


#### 112-113

In [9]:
#tenure也standardize
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [10]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [11]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_112.csv')

63/63 [==============================] - 2s 2ms/step


In [12]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_113.csv')

63/63 [==============================] - 0s 2ms/step


#### 114-115

In [20]:
#tenure也normalize
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [22]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [24]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_114.csv')

63/63 [==============================] - 0s 2ms/step


In [23]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_115.csv')

63/63 [==============================] - 0s 2ms/step


#### 116-117

In [31]:
#only normalize 3 col
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [32]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [33]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_116.csv')

63/63 [==============================] - 0s 2ms/step


In [34]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_117.csv')

63/63 [==============================] - 0s 2ms/step


#### 118-122

In [40]:
#only standardize 3 col
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [41]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [42]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_118.csv')

63/63 [==============================] - 0s 2ms/step


In [43]:
ypred = np.where(model_dnn.predict(test_X)>0.46,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_119.csv')

63/63 [==============================] - 0s 2ms/step


In [44]:
ypred = np.where(model_dnn.predict(test_X)>0.42,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_120.csv')

63/63 [==============================] - 0s 2ms/step


In [45]:
ypred = np.where(model_dnn.predict(test_X)>0.39,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_121.csv')

63/63 [==============================] - 0s 2ms/step


In [46]:
ypred = np.where(model_dnn.predict(test_X)>0.36,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_122.csv')

63/63 [==============================] - 0s 2ms/step


#### 123-

In [55]:
#only standardize 4 col
model_dnn=tf.keras.Sequential()

model_dnn.add(tf.keras.layers.Dense(25,activation='relu',input_dim=12))
model_dnn.add(tf.keras.layers.Dropout(0.5))
model_dnn.add(tf.keras.layers.Dense(10,activation='relu'))
model_dnn.add(tf.keras.layers.Dropout(0.3))
model_dnn.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=['accuracy'])

In [56]:
model_dnn.fit(train_X,train_Y,epochs=200,verbose=0)

In [53]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_123.csv')

63/63 [==============================] - 0s 2ms/step


In [54]:
ypred = np.where(model_dnn.predict(test_X)>0.44,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_124.csv')

63/63 [==============================] - 0s 2ms/step


In [57]:
ypred = np.where(model_dnn.predict(test_X)>0.5,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_125.csv')

63/63 [==============================] - 0s 2ms/step


In [58]:
ypred = np.where(model_dnn.predict(test_X)>0.45,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_126.csv')

63/63 [==============================] - 0s 2ms/step


In [61]:
ypred = np.where(model_dnn.predict(test_X)>0.40,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_127.csv')

63/63 [==============================] - 0s 2ms/step


In [62]:
ypred = np.where(model_dnn.predict(test_X)>0.35,1,0)
RowNumber = pd.DataFrame(RowNumber)
Y_pred = pd.DataFrame(ypred)
upload = pd.concat([RowNumber,Y_pred], axis=1)
upload.rename(columns={0: 'Exited'}, inplace=True)
upload.to_csv('upload_128.csv')

63/63 [==============================] - 0s 2ms/step
